In [1]:
import pymongo
import json
import pymongo
from netaddr import IPNetwork

def update_mongodb_document(json_data):
    # Connect to the MongoDB database
    client = pymongo.MongoClient(mongodb)
    db = client["config"]
    collection = db["tenant"]

    # Convert the JSON object to a Python dictionary
    data = json.loads(json_data)

    # Get the ID of the document to update
    document_id = data["_id"]

    # Get the fields to update
    update_fields = {}
    for key in data.keys():
        if key != "_id":
            update_fields[key] = data[key]

    # Update the MongoDB document with the specified fields
    collection.update_one({"_id": document_id}, {"$set": update_fields})


In [ ]:
def create_link(linkid, device1id, device2id, linkcost, underlay1id, underlay2id):
    """this function takes a link data and assignes a /31 Ip address to the link and saves all data in database"""
    # Connect to MongoDB
    client = pymongo.MongoClient('mongodb://localhost:27017/')
    db = client['ip_assignments']
    collection = db['sdwan_p2p']

    # Find an available /31 subnet
    subnet = None
    for cidr in IPNetwork('169.254.0.0/16').subnet(31):
        if not collection.find_one({'subnet': str(cidr)}):
            subnet = cidr
            break

    if not subnet:
        raise Exception('No available /31 subnets found')

    # Update MongoDB document with link information
    result = collection.update_one({'subnet': str(subnet)}, {'$set': {
        'linkid': linkid,
        'device1name': device1name,
        'device2name': device2name,
        'underlay1id': underlay1id,
        'underlay2id': underlay2id,
        'linkcost': linkcost,
    }}, upsert=True)

    # Get the inserted document's _id field
    document_id = result.upserted_id['_id']

    # Return the assigned subnet and document_id
    return str(subnet), str(document_id)